In [42]:
import pandas as pd
import pickle
import numpy as np
from json import loads
import requests
import re

This notebook includes the work to clean the food access data and merge it with the cleaned education dataset prior to analysis. 

# Load dataset

In [43]:
food_df = pickle.load(open('food_df.pkl', 'rb'))

# Initial Feature Selection

The food access atlas has a huge number of features to work from. As a start, I'll see if any are missing too many values for them to be useful. I'll pull a list of any column that has more than half of the data points missing

In [44]:
food_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72531 entries, 0 to 72530
Columns: 147 entries, CensusTract to TractSNAP
dtypes: float64(126), int64(19), object(2)
memory usage: 81.3+ MB


In [45]:
drop_cols = list(food_df.isnull().sum()[food_df.isnull().sum()> (len(food_df)//2)].index)

drop_cols

['lapop10',
 'lapop10share',
 'lalowi10',
 'lalowi10share',
 'lakids10',
 'lakids10share',
 'laseniors10',
 'laseniors10share',
 'lawhite10',
 'lawhite10share',
 'lablack10',
 'lablack10share',
 'laasian10',
 'laasian10share',
 'lanhopi10',
 'lanhopi10share',
 'laaian10',
 'laaian10share',
 'laomultir10',
 'laomultir10share',
 'lahisp10',
 'lahisp10share',
 'lahunv10',
 'lahunv10share',
 'lasnap10',
 'lasnap10share',
 'lapop20',
 'lapop20share',
 'lalowi20',
 'lalowi20share',
 'lakids20',
 'lakids20share',
 'laseniors20',
 'laseniors20share',
 'lawhite20',
 'lawhite20share',
 'lablack20',
 'lablack20share',
 'laasian20',
 'laasian20share',
 'lanhopi20',
 'lanhopi20share',
 'laaian20',
 'laaian20share',
 'laomultir20',
 'laomultir20share',
 'lahisp20',
 'lahisp20share',
 'lahunv20',
 'lahunv20share',
 'lasnap20',
 'lasnap20share']

Looking through the list of columns and comparing to the data description, it looks like all of these columns are population counts and percentages for various groups (e.g. lapop10 is the population count of individuals in the tract who live more than 10 miles from a supermarket). For this analysis, I'm not focused on populations, just whether the tract as a whole is low access. The dataset already has other features that flag a tract as low access or not under the USDA definitions, so I can drop these features and just use the USDA flags.

In [46]:
len(drop_cols)

52

In [47]:
# The columns in question are all next to eachother, dropping the section that contains these columns

food_df.drop(columns=food_df.iloc[:,31:-12].columns, inplace=True)

food_df.columns

Index(['CensusTract', 'State', 'County', 'Urban', 'Pop2010', 'OHU2010',
       'GroupQuartersFlag', 'NUMGQTRS', 'PCTGQTRS', 'LILATracts_1And10',
       'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle',
       'HUNVFlag', 'LowIncomeTracts', 'PovertyRate', 'MedianFamilyIncome',
       'LA1and10', 'LAhalfand10', 'LA1and20', 'LATracts_half', 'LATracts1',
       'LATracts10', 'LATracts20', 'LATractsVehicle_20', 'LAPOP1_10',
       'LAPOP05_10', 'LAPOP1_20', 'LALOWI1_10', 'LALOWI05_10', 'LALOWI1_20',
       'TractLOWI', 'TractKids', 'TractSeniors', 'TractWhite', 'TractBlack',
       'TractAsian', 'TractNHOPI', 'TractAIAN', 'TractOMultir',
       'TractHispanic', 'TractHUNV', 'TractSNAP'],
      dtype='object')

Looking at the remaining features with null values, they appear to all be either the same type of feature I dropped previously, or features that I don't need for the analysis. For instance the TractXXX features all have the count of the population that are members of that specific demographic. However, since I'm looking at school level data, I'm more concerned with the demographics of the students rather than the population as a whole. Therefore, I will just drop all of these features from the dataset. 

In [48]:
# Set display output
pd.options.display.max_rows = None

# Look at null values
food_df.isnull().sum()[food_df.isnull().sum() > 0]

NUMGQTRS                 25
PCTGQTRS                 25
PovertyRate               3
MedianFamilyIncome      748
LAPOP1_10             29957
LAPOP05_10            14540
LAPOP1_20             35914
LALOWI1_10            29957
LALOWI05_10           14540
LALOWI1_20            35914
TractLOWI                 4
TractKids                 4
TractSeniors              4
TractWhite                4
TractBlack                4
TractAsian                4
TractNHOPI                4
TractAIAN                 4
TractOMultir              4
TractHispanic             4
TractHUNV                 4
TractSNAP                 4
dtype: int64

In [49]:
# Reset display parameters
pd.options.display.max_rows = 20

# Create list of all columns with missing values
missing_vals = list(food_df.isnull().any()[food_df.isnull().any() == True].index)

# Drop columns
food_df.drop(columns=missing_vals, inplace=True)

In [50]:
food_df.columns

Index(['CensusTract', 'State', 'County', 'Urban', 'Pop2010', 'OHU2010',
       'GroupQuartersFlag', 'LILATracts_1And10', 'LILATracts_halfAnd10',
       'LILATracts_1And20', 'LILATracts_Vehicle', 'HUNVFlag',
       'LowIncomeTracts', 'LA1and10', 'LAhalfand10', 'LA1and20',
       'LATracts_half', 'LATracts1', 'LATracts10', 'LATracts20',
       'LATractsVehicle_20'],
      dtype='object')

I've now managed to get the features down from 147 to 21, a much more manageable number. There are a few more columns that I can drop since they will be unecessary: OHU2010 (total number of houses), Urban, and Pop2010.

In [51]:
food_df.drop(columns=['OHU2010', 'Urban', 'Pop2010'], inplace=True)

In [52]:
food_df.shape

(72531, 18)

In [53]:
# Validate that all null values have been removed

food_df.isnull().sum()[food_df.isnull().sum() > 0]

Series([], dtype: int64)

# Census Tract Feature Engineering

In [54]:
# Load edu df dataset
edu_df = pickle.load(open('edu_df.pkl', 'rb'))

Census tract codes are made up of 3 components: a 2 digit state code, a 3 digit county code, and a 6 digit tract id. The edu dataframe 'tract' column only includes the 6 digit tract id, but the 'county_code' column contains the other digits required. 

Taking a look at the different ids, I can see that the length for tract ids run from 3 to 6, the length for county code is 4 to 5 and the length of CensusTract runs from 10 to 11. That looks like all the codes have leading 0's removed so we'll need to add those back on to the 'tract' id to ensure the appropriate length.

source: https://transition.fcc.gov/form477/Geo/more_about_census_tracts.pdf

In [55]:
min_len = np.inf
max_len = 0
for i in edu_df['tract'].unique():
    item = str(int(i))
    if len(item) < min_len:
        min_len = len(item)
    if len(item) > max_len:
        max_len = len(item)

print(min_len, max_len)

min_len = np.inf
max_len = 0
for i in edu_df['county_code'].unique():
    item = str(int(i))
    if len(item) < min_len:
        min_len = len(item)
    if len(item) > max_len:
        max_len = len(item)

print(min_len, max_len)

print(len(str(food_df.CensusTract.unique().min())),
len(str(food_df.CensusTract.unique().max())))

for i in food_df['CensusTract'].unique():
    if len(str(i)) == 11:
        if str(i).split()[0] == 0:
            print('True')

3 6
4 5
10 11


In [56]:
# Note that tracts can have sub-categories marked as a decimal point, which is why the tract id is listed as a float. But a quick check verifies that there are no subcategories included

for i in edu_df['tract']:
    if i > int(i):
        print('True')

In [57]:
# Create a new column in the edu_df combining the county code and tract id
def census_tract (row):
    tract = str(int(row['tract']))
    code = str(row['county_code'])
    while len(tract) < 6:
        tract = '0' + tract
    
    census = code + tract
    return census

edu_df['CensusTract'] = edu_df.apply(census_tract, axis=1)
    

In [58]:
# Double check some samples to make sure it worked

edu_df.sample(10)

,ncessch,school_name,cohort_cat,cohort_num,street,city,state,zip,county_code,tract,school_level,title_i_status,charter,magnet,shared_time,virtual,lunch_program,urban_locale_cat,grad_rate,CensusTract
165514,481830012055,TRANSMOUNTAIN EARLY COLLEGE H S,homeless,3,9570 GATEWAY N BLVD EC,EL PASO,TX,79925,48141,101.0,3,5.0,0,0.0,-1.0,-1.0,1.0,Urban,64.97,48141000101
105738,320006000460,West Prepatory Academy at Charlles I West Hall...,white,1,2050 Sapphire Stone Ave.,Las Vegas,NV,89106,32003,302.0,3,5.0,0,0.0,0.0,0.0,1.0,Urban,85.12,32003000302
166639,482106002099,GONZALES H S,two_or_more,3,1801 N SARAH DEWITT DR,GONZALES,TX,78629,48177,300.0,3,5.0,0,0.0,-1.0,-1.0,4.0,Town,50.0,48177000300
137487,390487403386,Valley View High School,foster_care,1,6027 Farmersville Germantn Pik,Germantown,OH,45327,39113,165000.0,3,6.0,0,-1.0,0.0,0.0,1.0,Rural,49.78,39113165000
755,10078000321,Ranburne High School,white,51,21045 Main St,Ranburne,AL,36273,1029,959800.0,3,3.0,0,0.0,0.0,0.0,1.0,Rural,90,1029959800
121805,362037001879,NEW HARTFORD SENIOR HIGH SCHOOL,homeless,1,33 OXFORD RD,NEW HARTFORD,NY,13413,36065,25100.0,3,2.0,0,0.0,0.0,0.0,1.0,Suburban,51.89,36065025100
130982,381243000408,MCCLUSKY HIGH SCHOOL,total,7,219 Ave D W,McClusky,ND,58463,38083,960200.0,3,6.0,-2,-2.0,0.0,3.0,1.0,Rural,50,38083960200
166112,481970013114,MARINE CREEK COLLEGIATE H S,hispanic,70,4801 MARINE CREEK PKWY,FORT WORTH,TX,76107,48439,114008.0,3,5.0,0,0.0,-1.0,-1.0,1.0,Urban,95,48439114008
94555,272391001186,CENTRAL SENIOR HIGH,homeless,1,531 CENTRAL AVE,NORWOOD YOUNG AMERICA,MN,55368,27019,91201.0,3,2.0,0,0.0,0.0,0.0,1.0,Town,48.17,27019091201
158599,470369001524,Riverdale High School,api,27,802 Warrior DR,Murfreesboro,TN,37128,47149,41800.0,3,6.0,0,0.0,0.0,0.0,1.0,Urban,80,47149041800


In [59]:
# Convert column to integer type

edu_df['CensusTract'] = edu_df['CensusTract'].astype('int64')

# Join with Education Data

## First Joining Attempt

In [60]:
df = edu_df.merge(food_df, on= 'CensusTract', how='left')

After joining the datasets, there are 918 rows (associated with 189 schools) that didn't match with the food access dataset. Digging more into this, it appears that the census tracts are actually 2020 tracts in at least some cases not 2010, even though the file states that they are 2010. Since the school data we're using is 2019 it makes sense to match up to the 2020 data for those that didn't match properly. To do this, I'll need to use the Census Geocoder to look up each address and find the current census tract ID. 

source: https://geocoding.geo.census.gov/geocoder/

In [61]:
df.isnull().sum()[df.isnull().sum() > 0]

State                   918
County                  918
GroupQuartersFlag       918
LILATracts_1And10       918
LILATracts_halfAnd10    918
LILATracts_1And20       918
LILATracts_Vehicle      918
HUNVFlag                918
LowIncomeTracts         918
LA1and10                918
LAhalfand10             918
LA1and20                918
LATracts_half           918
LATracts1               918
LATracts10              918
LATracts20              918
LATractsVehicle_20      918
dtype: int64

In [62]:
def census_lookup(school):
    '''Input is a row from edu_df. Function parses the address and looks up the census tract ID from the census.gov geocoding website'''

    # Parse address into correct format
    old_tract = school['CensusTract']
    street = re.sub(pattern=r'["#"]', repl='', string=school['street'])
    geoStreet = '+'.join(street.split())
    geoCity = '+'.join(school['city'].split())
    geoState = school['state']
    geoZip= school['zip']

    # Make Request
    url = f'https://geocoding.geo.census.gov/geocoder/geographies/address?street={geoStreet}&city={geoCity}&state={geoState}&zip={geoZip}&benchmark=Public_AR_Current&vintage=Current_Current&layers=11&format=json'
    response = requests.get(url)

    # Load results from request
    results = loads(response.text)

    # Check if search returned results or not
    if len(results['result']['addressMatches']) > 0:
        # Get census tract ID
        new_tract = results['result']['addressMatches'][0]['geographies']['Census Tracts'][0]['GEOID']
    else:
        new_tract = 'No Match'
    
    return new_tract

In [63]:
# List of school IDs for schools that didn't have a match
missing_schools = list(df[df['State'].isnull()]['ncessch'].unique())
temp_dict = {}

for idx, id in enumerate(missing_schools):
    # Loop through each school and look up it's 2020 census tract ID if available
    school = edu_df[edu_df['ncessch'] == id].iloc[0]
    old_tract = edu_df[edu_df['ncessch'] == id].iloc[0]['CensusTract']
    new_tract = census_lookup(school)

    temp_dict[id] = [old_tract, new_tract]

    # Print the index of each ncessch to verify code is running properly and to debug as needed
    print(f'Index {idx} complete')

Index 0 complete
Index 1 complete
Index 2 complete
Index 3 complete
Index 4 complete
Index 5 complete
Index 6 complete
Index 7 complete
Index 8 complete
Index 9 complete
Index 10 complete
Index 11 complete
Index 12 complete
Index 13 complete
Index 14 complete
Index 15 complete
Index 16 complete
Index 17 complete
Index 18 complete
Index 19 complete
Index 20 complete
Index 21 complete
Index 22 complete
Index 23 complete
Index 24 complete
Index 25 complete
Index 26 complete
Index 27 complete
Index 28 complete
Index 29 complete
Index 30 complete
Index 31 complete
Index 32 complete
Index 33 complete
Index 34 complete
Index 35 complete
Index 36 complete
Index 37 complete
Index 38 complete
Index 39 complete
Index 40 complete
Index 41 complete
Index 42 complete
Index 43 complete
Index 44 complete
Index 45 complete
Index 46 complete
Index 47 complete
Index 48 complete
Index 49 complete
Index 50 complete
Index 51 complete
Index 52 complete
Index 53 complete
Index 54 complete
Index 55 complete
In

In [64]:
# Check how many values did not have a match
count = 0
for item in temp_dict.keys():
    if temp_dict[item][1] == 'No Match':
        count += 1

print(f'Out of {len(temp_dict)} census tracts, {count} were not found in the lookup')

Out of 189 census tracts, 159 were not found in the lookup


In reviewing the matches, it looks like I only matched 30 census tracts using the geocoder site. After a spot check it looks like most schools have the same state code (72) which represents Puerto Rico. A quick loop confirms that all but 1 tract where I could not find a match were in Puerto Rico. The reaming tract had 2 schools in NY, neither of which I could find through manual searching. 

In [65]:
# Check how many of the schools missing a match have the state code 72
missing_vals = []
count = 0
for item in temp_dict.keys():
    if temp_dict[item][1] == 'No Match':
        missing_vals.append(item)

for item in missing_vals:
    if str(item)[:2] == '72':
        count+= 1

print(count)

158


Because so many of the schools in Puerto Rico are missing this information, I will drop all of the Puerto Rico schools from the dataset to avoid skewing the data with an unrepresentative sample. 

In [66]:
# Drop all schools with a state of PR
edu_df = edu_df[edu_df['state'] != 'PR']

## Update edu_df with correct census tract id

In [67]:
# Create dictionary of old tract id and their corrected one
tract_changes = pd.DataFrame.from_dict(temp_dict, columns=['old_tract', 'new_tract'], orient='index')

tract_changes

,old_tract,new_tract
40052003063,4019470400,04019005200
40089803393,4013980600,04013422647
60211511108,6071980100,06071980100
80195000010,8001988700,08001988700
80336006527,8031980100,08031980100
...,...,...
720003002064,72059740400,No Match
720003002066,72123952800,No Match
720003002069,72031050600,No Match
720003002073,72031051102,No Match


In [68]:
# Update all tracts

def update_tract(x):
    if x in list(tract_changes['old_tract']):
        tract = tract_changes[tract_changes['old_tract'] == x].new_tract.values[0]
        return tract
    else:
        return x
    

edu_df['CensusTract2'] = edu_df['CensusTract'].apply(update_tract)

# Spot check one of the values
edu_df[edu_df['CensusTract'] == 4019470400]



,ncessch,school_name,cohort_cat,cohort_num,street,city,state,zip,county_code,tract,...,title_i_status,charter,magnet,shared_time,virtual,lunch_program,urban_locale_cat,grad_rate,CensusTract,CensusTract2
6209,40052003063,Ajo High School,total,26,111 N WELL RD,AJO,AZ,85321,4019,470400.0,...,5.0,0,-1.0,0.0,0.0,1.0,Town,80,4019470400,04019005200
6210,40052003063,Ajo High School,disability,6,111 N WELL RD,AJO,AZ,85321,4019,470400.0,...,5.0,0,-1.0,0.0,0.0,1.0,Town,50,4019470400,04019005200
6211,40052003063,Ajo High School,econ_disadvantaged,16,111 N WELL RD,AJO,AZ,85321,4019,470400.0,...,5.0,0,-1.0,0.0,0.0,1.0,Town,80,4019470400,04019005200
6212,40052003063,Ajo High School,limited_english,3,111 N WELL RD,AJO,AZ,85321,4019,470400.0,...,5.0,0,-1.0,0.0,0.0,1.0,Town,53.22,4019470400,04019005200
6213,40052003063,Ajo High School,am_indian/ak_native,3,111 N WELL RD,AJO,AZ,85321,4019,470400.0,...,5.0,0,-1.0,0.0,0.0,1.0,Town,72.53,4019470400,04019005200
6214,40052003063,Ajo High School,api,1,111 N WELL RD,AJO,AZ,85321,4019,470400.0,...,5.0,0,-1.0,0.0,0.0,1.0,Town,56.0,4019470400,04019005200
6215,40052003063,Ajo High School,hispanic,22,111 N WELL RD,AJO,AZ,85321,4019,470400.0,...,5.0,0,-1.0,0.0,0.0,1.0,Town,80,4019470400,04019005200


In [69]:
# Verify that the only schools with no match were the ones I identified. 
edu_df[edu_df['CensusTract2'] == 'No Match']

,ncessch,school_name,cohort_cat,cohort_num,street,city,state,zip,county_code,tract,...,title_i_status,charter,magnet,shared_time,virtual,lunch_program,urban_locale_cat,grad_rate,CensusTract,CensusTract2
121682,361992001819,MORRISVILLE MIDDLE SCHOOL HIGH SCHOOL,total,54,5061 FEARON RD,MORRISVILLE,NY,13408,36053,940600.0,...,5.0,0,0.0,0.0,0.0,1.0,Rural,84.5,36053940600,No Match
121683,361992001819,MORRISVILLE MIDDLE SCHOOL HIGH SCHOOL,disability,12,5061 FEARON RD,MORRISVILLE,NY,13408,36053,940600.0,...,5.0,0,0.0,0.0,0.0,1.0,Rural,50,36053940600,No Match
121684,361992001819,MORRISVILLE MIDDLE SCHOOL HIGH SCHOOL,econ_disadvantaged,34,5061 FEARON RD,MORRISVILLE,NY,13408,36053,940600.0,...,5.0,0,0.0,0.0,0.0,1.0,Rural,84.5,36053940600,No Match
121685,361992001819,MORRISVILLE MIDDLE SCHOOL HIGH SCHOOL,api,3,5061 FEARON RD,MORRISVILLE,NY,13408,36053,940600.0,...,5.0,0,0.0,0.0,0.0,1.0,Rural,57.0,36053940600,No Match
121686,361992001819,MORRISVILLE MIDDLE SCHOOL HIGH SCHOOL,black,2,5061 FEARON RD,MORRISVILLE,NY,13408,36053,940600.0,...,5.0,0,0.0,0.0,0.0,1.0,Rural,63.27,36053940600,No Match
121687,361992001819,MORRISVILLE MIDDLE SCHOOL HIGH SCHOOL,two_or_more,1,5061 FEARON RD,MORRISVILLE,NY,13408,36053,940600.0,...,5.0,0,0.0,0.0,0.0,1.0,Rural,51.67,36053940600,No Match
121688,361992001819,MORRISVILLE MIDDLE SCHOOL HIGH SCHOOL,white,48,5061 FEARON RD,MORRISVILLE,NY,13408,36053,940600.0,...,5.0,0,0.0,0.0,0.0,1.0,Rural,84.5,36053940600,No Match
123386,362814003801,STOCKBRIDGE VALLEY CENTRAL SCHOOL,total,29,6011 WILLIAMS RD,MUNNSVILLE,NY,13409,36053,940600.0,...,3.0,0,0.0,0.0,0.0,1.0,Rural,80,36053940600,No Match
123387,362814003801,STOCKBRIDGE VALLEY CENTRAL SCHOOL,disability,5,6011 WILLIAMS RD,MUNNSVILLE,NY,13409,36053,940600.0,...,3.0,0,0.0,0.0,0.0,1.0,Rural,56.14,36053940600,No Match
123388,362814003801,STOCKBRIDGE VALLEY CENTRAL SCHOOL,econ_disadvantaged,6,6011 WILLIAMS RD,MUNNSVILLE,NY,13409,36053,940600.0,...,3.0,0,0.0,0.0,0.0,1.0,Rural,50,36053940600,No Match


In [70]:
# Dropping the items with no match

edu_df = edu_df[edu_df['CensusTract2'] != 'No Match']

# Removing old column and renaming new column

edu_df.drop(columns=['CensusTract'], inplace=True)

edu_df.rename(columns={'CensusTract2': 'CensusTract'}, inplace=True)

In [71]:
# Convert to an integer

edu_df['CensusTract'] = edu_df['CensusTract'].astype('int64')

## Second Joining Attempt

In [72]:
df = edu_df.merge(food_df, on= 'CensusTract', how='left')

This has now gotten the missing matches down to 60 rows and 7 schools

In [73]:
print(df.isnull().sum()[df.isnull().sum()> 0])

print(df[df['State'].isnull()]['ncessch'].nunique())

State                   60
County                  60
GroupQuartersFlag       60
LILATracts_1And10       60
LILATracts_halfAnd10    60
LILATracts_1And20       60
LILATracts_Vehicle      60
HUNVFlag                60
LowIncomeTracts         60
LA1and10                60
LAhalfand10             60
LA1and20                60
LATracts_half           60
LATracts1               60
LATracts10              60
LATracts20              60
LATractsVehicle_20      60
dtype: int64
7


Looking up the remaining schools, I can't find any information for these census tracts. Because it is only a handful of schools and they are spread out across the country, I wil just drop them from the data.

In [74]:
for i in list(df[df['State'].isnull()]['ncessch'].unique())[:7]:
    print(df[df['ncessch'] == i]['school_name'].unique(),df[df['ncessch'] == i]['street'].unique(), df[df['ncessch'] == i]['city'].unique(), df[df['ncessch'] == i]['state'].unique(), df[df['ncessch'] == i]['CensusTract'].unique())

['BASIS Mesa'] ['5010 S EASTMARK PKWY'] ['MESA'] ['AZ'] [4013422647]
['Public Safety Academy'] ['1482 E. Enterprise Dr.'] ['San Bernardino'] ['CA'] [6071980100]
['ADAMS CITY HIGH SCHOOL'] ['7200 QUEBEC PARKWAY'] ['COMMERCE CITY'] ['CO'] [8001988700]
['HIGH TECH EARLY COLLEGE'] ['11200 E 45TH AVE'] ['DENVER'] ['CO'] [8031980100]
['The Founders Academy Charter School (H)'] ['5 Perimeter Rd'] ['Manchester'] ['NH'] [33011980101]
['QUEENS HIGH SCHOOL FOR THE SCIENCES AT YORK COLLEGE'] ['94-50 159TH ST'] ['JAMAICA'] ['NY'] [36081024600]
['Midlands Middle College'] ['1260 Lexington Drive'] ['West Columbia'] ['SC'] [45063980100]


In [75]:
# Drop any columns that didn't match
df = df[df['State'].isnull() == False]

# double check that all null values are removed
df.isnull().sum()[df.isnull().sum() > 0]

Series([], dtype: int64)

In [76]:
len(edu_df)

145324

# Filter Dataset

Finally, I have joined all of the data together and removed all missing values. There is a little more cleaning that I can do before I start the analysis. 

1) The GroupQuartersFlag indicates that more than 67% of the population lives in 'group quarters' which are communal living situations like military barracks, employee housing, etc. Since these situations are less common and introduce additonal factors that may skew our results (for instance military barracks will provide food for those living there, making the proximity of a grocery store irrelevant) I'll drop any tracts where this is true.

2) Some features such as the address of the school and the school name were only needed for matching and joining purposes. I can drop these columns to make the analysis simpler.

3) I can reorganize the columns to read a little easier

4) I can set the correct types for each of the features

In [77]:
# Remove group quarters observations
df = df[df.GroupQuartersFlag == 0]

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144951 entries, 0 to 145323
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ncessch               144951 non-null  int64  
 1   school_name           144951 non-null  object 
 2   cohort_cat            144951 non-null  object 
 3   cohort_num            144951 non-null  int64  
 4   street                144951 non-null  object 
 5   city                  144951 non-null  object 
 6   state                 144951 non-null  object 
 7   zip                   144951 non-null  object 
 8   county_code           144951 non-null  int64  
 9   tract                 144951 non-null  float64
 10  school_level          144951 non-null  int64  
 11  title_i_status        144951 non-null  float64
 12  charter               144951 non-null  int64  
 13  magnet                144951 non-null  float64
 14  shared_time           144951 non-null  float64
 15  

# Clean and Save Final Dataframe

In [79]:
# Drop unecessary columns and reorganize
df = df[['ncessch', 'CensusTract', 'cohort_num','cohort_cat', 'school_level', 'charter', 'title_i_status', 'lunch_program', 'magnet', 'shared_time', 'virtual', 'LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle',
         'LA1and10', 'LAhalfand10', 'LA1and20', 'LATractsVehicle_20','grad_rate']]

In [80]:
# Check columns

df.columns

Index(['ncessch', 'CensusTract', 'cohort_num', 'cohort_cat', 'school_level',
       'charter', 'title_i_status', 'lunch_program', 'magnet', 'shared_time',
       'virtual', 'LILATracts_1And10', 'LILATracts_halfAnd10',
       'LILATracts_1And20', 'LILATracts_Vehicle', 'LA1and10', 'LAhalfand10',
       'LA1and20', 'LATractsVehicle_20', 'grad_rate'],
      dtype='object')

In [81]:
# Update columns to appropriate type

# The only numerical columns are grad_rate and cohort_num, so I'll just change everything to a category and then update the two relevant columns to integer types
df = df.astype('category')

df[['grad_rate', 'cohort_num']] = df[['grad_rate', 'cohort_num']].astype('int64')

df.dtypes


ncessch                 category
CensusTract             category
cohort_num                 int64
cohort_cat              category
school_level            category
charter                 category
title_i_status          category
lunch_program           category
magnet                  category
shared_time             category
virtual                 category
LILATracts_1And10       category
LILATracts_halfAnd10    category
LILATracts_1And20       category
LILATracts_Vehicle      category
LA1and10                category
LAhalfand10             category
LA1and20                category
LATractsVehicle_20      category
grad_rate                  int64
dtype: object

In [82]:
df.to_pickle(open('final_df.pkl', 'wb'))